In [1]:

import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Bidirectional, Embedding, Dense, Dropout, BatchNormalization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
data = pd.read_csv(r'/content/IMDB Dataset.csv', on_bad_lines='skip')


In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define the text cleaning function
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<br\s*/?>', ' ', text)  # Remove <br> tags
    text = re.sub(r'<.*?>', ' ', text)      # Remove other HTML tags
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove non-alphabetic characters
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Split into words
    words = text.split()

    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)

    return cleaned_text

# Apply the cleaning function to the review column
data['cleaned_review'] = data['review'].apply(clean_text)

print(data[['review', 'cleaned_review']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one reviewer mentioned watching oz episode you...  
1  wonderful little production filming technique ...  
2  thought wonderful way spend time hot summer we...  
3  basically there family little boy jake think t...  
4  petter matteis love time money visually stunni...  


In [6]:
X=data['cleaned_review']
X[0]

'one reviewer mentioned watching oz episode youll hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home manyaryans muslim gangsta latino christian italian irish moreso scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show wouldnt dare forget pretty picture painted mainstream audience forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard wholl sold nickel inmate wholl kill order get away well mannered middle class inmate turned prison bitch due la

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=1422)

In [8]:
y=data['sentiment']
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [10]:
maxlength=1422
voc_size=30000

In [11]:
y = y.map({'negative': 0, 'positive': 1})
y = y.values

In [12]:
from sklearn.model_selection import train_test_split

# Assuming embedded_doc contains your padded sequences and y contains the labels

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes to confirm the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (40000, 1422)
X_test shape: (10000, 1422)
y_train shape: (40000,)
y_test shape: (10000,)


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    mode='max',
    restore_best_weights=True
)


from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, BatchNormalization, Dropout, Dense
from tensorflow.keras.regularizers import l2  # Import the L2 regularization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=30000, output_dim=300, input_length=1422))

# First Bidirectional LSTM layer with dropout
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Second LSTM layer without return_sequences (no 3D output)
model.add(LSTM(32, return_sequences=False, kernel_regularizer=l2(0.01)))  # return_sequences=False to get 2D output
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Dense layers
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))  # Additional dropout
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
history = model.fit(
    X_train,  # Training data features
    y_train,  # Training data labels
    epochs=20,  # Number of epochs to train the model
    batch_size=64,  # Number of samples per gradient update
    validation_data=(X_test, y_test),  # Validation data (features and labels)
    callbacks=[model_checkpoint, early_stopping,reduce_lr]  # List of callbacks to apply during training
)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 125s 182ms/step - accuracy: 0.5118 - loss: 1.8279 - val_accuracy: 0.8479 - val_loss: 0.5610 - learning_rate: 0.0010
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 184ms/step - accuracy: 0.8857 - loss: 0.4623 - val_accuracy: 0.8891 - val_loss: 0.3180 - learning_rate: 0.0010
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 181ms/step - accuracy: 0.9451 - loss: 0.2069 - val_accuracy: 0.8685 - val_loss: 0.3969 - learning_rate: 0.0010
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 114s 183ms/step - accuracy: 0.9678 - loss: 0.1292 - val_accuracy: 0.8833 - val_loss: 0.3453 - learning_rate: 0.0010
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 114s 183ms/step - accuracy: 0.9810 - loss: 0.0900 - val_accuracy: 0.8696 - val_loss: 0.4677 - learning_rate: 0.0010
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 113s 180ms/step - accuracy: 0.9912 - loss: 0.0518 - val_accuracy: 0.8798 - val_loss: 0.4931 - learning_rate: 2.0000e-04


In [16]:
model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step


array([[0.1011534 ],
       [0.9157767 ],
       [0.03423855],
       ...,
       [0.73009795],
       [0.03280244],
       [0.7730254 ]], dtype=float32)

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 55ms/step - accuracy: 0.8880 - loss: 0.3177
Loss: 0.31798097491264343
Accuracy: 0.8891000151634216


In [18]:
# Save the trained model in H5 format.
model.save("movie_review.h5")

# Save the tokenizer using pickle.
import pickle
with open("tokenizer_1.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
from google.colab import files
files.download(r"/content/movie_tokenizer.pickle")
files.downlaod(r"/content/movie_review.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>